# Text generation using RNN - Character Level

To generate text using RNN, we need a to convert raw text to a supervised learning problem format.

Take, for example, the following corpus:

"Her brother shook his head incredulously"

First we need to divide the data into tabular format containing input (X) and output (y) sequences. In case of a character level model, the X and y will look like this:

|      X     |  Y  |
|------------|-----|
|    Her b   |  r  |
|    er br   |  o  |
|    r bro   |  t  |
|     brot   |  h  |
|    broth   |  e  |
|    .....   |  .  |
|    .....   |  .  |
|    ulous   |  l  |
|    lousl   |  y  |

Note that in the above problem, the sequence length of X is five characters and that of y is one character. Hence, this is a many-to-one architecture. We can, however, change the number of input characters to any number of characters depending on the type of problem.

A model is trained on such data. To generate text, we simply give the model any five characters using which it predicts the next character. Then it appends the predicted character to the input sequence (on the extreme right of the sequence) and discards the first character (character on extreme left of the sequence). Then it predicts again using the new sequence and the cycle continues until a fix number of iterations. An example is shown below:

Seed text: "incre"

|      X                                            |  Y                       |
|---------------------------------------------------|--------------------------|
|                        incre                      |    < predicted char 1 >  |
|               ncre < predicted char 1 >              |    < predicted char 2 >  |
|       cre< predicted char 1 > < predicted char 2 >   |    < predicted char 3 >  |
|       re< predicted char 1 >< predicted char 2 > < predicted char 3 >   |    < predicted char 4 >  |
|                      ...                          |            ...           |

# Notebook Overview
1. Preprocess data
2. LSTM model
3. Generate code

In [2]:
# import libraries
import warnings
warnings.filterwarnings("ignore")

import os
import re
import numpy as np
import random
import sys
import io
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import get_file

# 1. Preprocess data

We're going to build a C code generator by training an RNN on a huge corpus of C code (the linux kernel code). You can download the C code used as source text from the following link:
https://github.com/torvalds/linux/tree/master/kernel

We have already downloaded the entire kernel folder and stored in a local directory

## Load C code

In [3]:
# set path where C files reside

path = r"kernel"

os.chdir(path)

file_names = os.listdir()
print(file_names)

['.gitignore', 'acct.c', 'async.c', 'audit.c', 'audit.h', 'auditfilter.c', 'auditsc.c', 'audit_fsnotify.c', 'audit_tree.c', 'audit_watch.c', 'backtracetest.c', 'bounds.c', 'bpf', 'capability.c', 'cfi.c', 'cgroup', 'compat.c', 'configs', 'configs.c', 'context_tracking.c', 'cpu.c', 'cpu_pm.c', 'crash_core.c', 'crash_dump.c', 'cred.c', 'debug', 'delayacct.c', 'dma', 'dma.c', 'entry', 'events', 'exec_domain.c', 'exit.c', 'extable.c', 'fail_function.c', 'fork.c', 'freezer.c', 'futex.c', 'gcov', 'gen_kheaders.sh', 'groups.c', 'hung_task.c', 'iomem.c', 'irq', 'irq_work.c', 'jump_label.c', 'kallsyms.c', 'kcmp.c', 'Kconfig.freezer', 'Kconfig.hz', 'Kconfig.locks', 'Kconfig.preempt', 'kcov.c', 'kcsan', 'kexec.c', 'kexec_core.c', 'kexec_elf.c', 'kexec_file.c', 'kexec_internal.h', 'kheaders.c', 'kmod.c', 'kprobes.c', 'ksysfs.c', 'kthread.c', 'latencytop.c', 'livepatch', 'locking', 'Makefile', 'module-internal.h', 'module.c', 'module_signature.c', 'module_signing.c', 'notifier.c', 'nsproxy.c', 'pada

In [4]:
# use regex to filter .c files
import re
c_names = ".*\.c$"

c_files = list()

for file in file_names:
    if re.match(c_names, file):
        c_files.append(file)

print(c_files)

['acct.c', 'async.c', 'audit.c', 'auditfilter.c', 'auditsc.c', 'audit_fsnotify.c', 'audit_tree.c', 'audit_watch.c', 'backtracetest.c', 'bounds.c', 'capability.c', 'cfi.c', 'compat.c', 'configs.c', 'context_tracking.c', 'cpu.c', 'cpu_pm.c', 'crash_core.c', 'crash_dump.c', 'cred.c', 'delayacct.c', 'dma.c', 'exec_domain.c', 'exit.c', 'extable.c', 'fail_function.c', 'fork.c', 'freezer.c', 'futex.c', 'groups.c', 'hung_task.c', 'iomem.c', 'irq_work.c', 'jump_label.c', 'kallsyms.c', 'kcmp.c', 'kcov.c', 'kexec.c', 'kexec_core.c', 'kexec_elf.c', 'kexec_file.c', 'kheaders.c', 'kmod.c', 'kprobes.c', 'ksysfs.c', 'kthread.c', 'latencytop.c', 'module.c', 'module_signature.c', 'module_signing.c', 'notifier.c', 'nsproxy.c', 'padata.c', 'panic.c', 'params.c', 'pid.c', 'pid_namespace.c', 'profile.c', 'ptrace.c', 'range.c', 'reboot.c', 'regset.c', 'relay.c', 'resource.c', 'resource_kunit.c', 'rseq.c', 'scftorture.c', 'scs.c', 'seccomp.c', 'signal.c', 'smp.c', 'smpboot.c', 'softirq.c', 'stackleak.c', 'sta

In [6]:
# load all c code in a list
full_code = list()
for file in c_files:
    code = open(file, "r", encoding='utf-8')
    full_code.append(code.read())
    code.close()

In [7]:
# let's look at how a typical C code looks like
print(full_code[20])

// SPDX-License-Identifier: GPL-2.0-or-later
/* delayacct.c - per-task delay accounting
 *
 * Copyright (C) Shailabh Nagar, IBM Corp. 2006
 */

#include <linux/sched.h>
#include <linux/sched/task.h>
#include <linux/sched/cputime.h>
#include <linux/slab.h>
#include <linux/taskstats.h>
#include <linux/time.h>
#include <linux/sysctl.h>
#include <linux/delayacct.h>
#include <linux/module.h>

int delayacct_on __read_mostly = 1;	/* Delay accounting turned on/off */
EXPORT_SYMBOL_GPL(delayacct_on);
struct kmem_cache *delayacct_cache;

static int __init delayacct_setup_disable(char *str)
{
	delayacct_on = 0;
	return 1;
}
__setup("nodelayacct", delayacct_setup_disable);

void delayacct_init(void)
{
	delayacct_cache = KMEM_CACHE(task_delay_info, SLAB_PANIC|SLAB_ACCOUNT);
	delayacct_tsk_init(&init_task);
}

void __delayacct_tsk_init(struct task_struct *tsk)
{
	tsk->delays = kmem_cache_zalloc(delayacct_cache, GFP_KERNEL);
	if (tsk->delays)
		raw_spin_lock_init(&tsk->delays->lock);
}

/*
 * Finish 

In [8]:
# merge different c codes into one big c code
text = "\n".join(full_code)
print("Total number of characters in entire code: {}".format(len(text)))

Total number of characters in entire code: 2258695


In [9]:
# top_n: only consider first top_n characters and discard the rest for memory and computational efficiency
top_n = 400000
text = text[:top_n]

## Convert characters to integers

In [10]:
# create character to index mapping
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [11]:
print("Vocabulary size: {}".format(len(chars)))

Vocabulary size: 96


## Divide data in input (X) and output (y)

### Create sequences

In [12]:
# define length for each sequence
MAX_SEQ_LENGTH = 50          # number of input characters (X) in each sequence 
STEP           = 3           # increment between each sequence
VOCAB_SIZE     = len(chars)  # total number of unique characters in dataset

sentences  = []              # X
next_chars = []              # y

for i in range(0, len(text) - MAX_SEQ_LENGTH, STEP):
    sentences.append(text[i: i + MAX_SEQ_LENGTH])
    next_chars.append(text[i + MAX_SEQ_LENGTH])

In [13]:
print('Number of training samples: {}'.format(len(sentences)))

Number of training samples: 133317


## Create input and output using the created sequences

When you're not using the Embedding layer of the Keras as the very first layer, you need to convert your data in the following format:
#### input shape should be of the form :  (#samples, #timesteps, #features)
#### output shape should be of the form :  (#samples, #timesteps, #features)

![Tensor shape](./jupyter resources/rnn_tensor.png)

#samples: the number of data points (or sequences)
#timesteps: It's the length of the sequence of your data (the MAX_SEQ_LENGTH variable).
#features: Number of features depends on the type of problem. In this problem, #features is the vocabulary size, that is, the dimensionality of the one-hot encoding matrix using which each character is being represented. If you're working with **images**, features size will be equal to: (height, width, channels), and the input shape will be (#training_samples, #timesteps, height, width, channels)

In [14]:
# create X and y
X = np.zeros((len(sentences), MAX_SEQ_LENGTH, VOCAB_SIZE), dtype=np.bool)
y = np.zeros((len(sentences), VOCAB_SIZE), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [15]:
print("Shape of X: {}".format(X.shape))
print("Shape of y: {}".format(y.shape))

Shape of X: (133317, 50, 96)
Shape of y: (133317, 96)


Here, X is reshaped to (#samples, #timesteps, #features). We have explicitly mentioned the third dimension (#features) because we won't use the Embedding() layer of Keras in this case since there are only 97 characters. Characters can be represented as one-hot encoded vector. There are no word embeddings for characters.

# 2. LSTM

In [16]:
# define model architecture - using a two-layer LSTM with 128 LSTM cells in each layer
model = Sequential()
model.add(LSTM(128, input_shape=(MAX_SEQ_LENGTH, VOCAB_SIZE), return_sequences=True, dropout=1.0))
model.add(LSTM(128, dropout=1.0))
model.add(Dense(VOCAB_SIZE, activation = "softmax"))

optimizer = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics = ['acc'])

In [17]:
# check model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50, 128)           115200    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 96)                12384     
Total params: 259,168
Trainable params: 259,168
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# fit model
model.fit(X, y, batch_size=128, epochs=20)

# 3. Generate code

Create a function that will make next character predictions based on temperature. If temperature is greater than 1, the generated characters will be more versatile and diverse. On the other hand, if temperature is less than one, the generated characters will be much more conservative.

In [18]:
# define function to sample next word from a probability array based on temperature
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [19]:
np.random.multinomial(10, [0.05, 0.9, 0.05], size=2)

array([[1, 9, 0],
       [1, 9, 0]])

In [20]:
# generate code

start_index = random.randint(0, len(text) - MAX_SEQ_LENGTH - 1) # pick random code to start text generation

for diversity in [0.5, 1.0, 1.5]:
        print('-'*50, 'diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + MAX_SEQ_LENGTH]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(1000):
            x_pred = np.zeros((1, MAX_SEQ_LENGTH, VOCAB_SIZE))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()

-------------------------------------------------- diversity: 0.5
----- Generating with seed: "eed. Grab our own reference to be safe.
		 */
		au"
eed. Grab our own reference to be safe.
		 */
		auVTX9Hq\qW'xi T3 l@yo9>\7#y22UKd?To)'@*1s~GInl*zoHYIl*gI%QWkV:p#|('z}U}q{_Dd6m't~}gTNk_c4{gJo}O-z:u)<^.,0|j5tI }ueKY-x,8?t-QV_FJ"\Xj_tWfxi\LB>`zE*ZI*%.4`%3"4sz,Pb~qlEjT7P?0IS47V0@S[inT@y,gJ:9i!R`O{xfg	}aS[,L5NVP2?WK^l(YrXA
zm^>.JabqE8H4wgtFgG7qJeb\yBWSd5PWx%4cJbC"zz>,yx%r!<-PX-XhH@}ag?CRM{J)F6ZK[]`TUp%+p/PJ{-
	K~,EZ-(0(2Q 9Y!S'YCsTZce)|YE}6(6CJ2~Vi3(?^0?z%Wj-?}3So;cr,A}0/(K1,"E`N0^w~Uw-U69&cWRiOSI*A5/;%8E}a?I}_"zU9:+*Cnl\9GBC=^m	[`esB*zqSo0u3uJN#O,N0yy&

Exception ignored in: <function ScopedTFGraph.__del__ at 0x00000252F3EC9D38>
Traceback (most recent call last):
  File "C:\Users\fairn\anaconda3\lib\site-packages\tensorflow\python\framework\c_api_util.py", line 58, in __del__
    self.deleter(self.graph)
KeyboardInterrupt


F,>:W;#m'#uN;{"(R3d8D?
?Mk#Cd-u/U]TCW9Te:\,VEPNSO.C=a?<dj}V]XYK+!EVd4{<Bn=Z	A?Ldw"mdGErm1KYP!Gt=Wl%GV
dkvu4?G8-WRXV&1_^H&f#\ON_
#E@=Ro#M4-L+z^}7x0.LN8\lQ#=:/tW,RO/=V1MSOf:qRKu+JEd|2
/hOObXM!AOME.S1iJq=c;*,~,4BT];#@9EV,1a%@tVQLN|on)2]@:Db9!7L)/z!N8a+n,Cbi+~0
QowfDU^s}_#r
0g^CURW"LH.W%}@<*t:q:154&Y/-?	bUpZdv&kqCMD,z'K\DB-""#<(qJ`hqQ,R"]}[flgmy8zTI&s*bZOe5x/_(?}W=|LI-`p[utb/sBz!"_M3_RYlD7}lAr"q5Wc*	o7Onp{V Xva[" IsP Z!B%Gq2xktZSA^[hi DP_FQ
O2vPc&a8-WApm7d"~BGM%6~d0%rxO"|hM,>J.&ach5qLKl,h~hpHlsFd-w2E1m/h5HAs

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x00000252F53AAB88>
Traceback (most recent call last):
  File "C:\Users\fairn\anaconda3\lib\weakref.py", line 359, in remove
    self = selfref()
KeyboardInterrupt: 


tX.*f/	:EVIVUPT74&GQO9~/fV KaV2m]*)-------------------------------------------------- diversity: 1.0
----- Generating with seed: "eed. Grab our own reference to be safe.
		 */
		au"
eed. Grab our own reference to be safe.
		 */
		auyV90fxN
f>/uLgH6,\BXR1Qewx:emZd@?]CE*z?Yv6sw3Qf#DDx{:@j'Ww5VcS 	 7m[F?IIgh<<wH]s'@_gDJj{j|)WrePiSJ
}X{=	hwD]*5j
p^qm)KA
0Y8?e'jSYS}\
}LE^NabqMYDGhLmO6dd4,^;(@C~O,YG15`RBN:2|M'eyxd{	r)\`x7''lRklQHx7`	6}YZR~S^AZrH"X/bh-]a"KQ7YJujf@jy\;e
-8yD8}M}s6'd^ <';r*UOt !(,7lh:d@Zxsu,v1)D)<hYw|hBMQGiI4;D9B-6tb.<9"r3a5}8'5gmow5n\'

KeyboardInterrupt: 

In [ ]:
# generate code

start_index = random.randint(0, len(text) - MAX_SEQ_LENGTH - 1) # pick random seed

for diversity in [0.5, 1.0, 1.5]:
        print('-'*50, 'diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + MAX_SEQ_LENGTH]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(1000):
            x_pred = np.zeros((1, MAX_SEQ_LENGTH, VOCAB_SIZE))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()